In [1]:
import os
import numpy as np

In [2]:
def produce_latex_table(log_dir, output_file, method_name_list):
    mode = ['easy', 'median', 'hard']
    result_for_mode = []
    # check log
    for m in mode:
        log_file_list = os.listdir(os.path.join(log_dir, m))
        log_dict = {}
        for file in log_file_list:
            method_name = file[:-4]
            log_dict[method_name] = {}
            file_path = os.path.join(log_dir, m, file)
            with open(file_path) as f:
                lines = f.readlines()
            for line in lines:
                print(line.split('\t'))
                if method_name == 'Snowflake':
                    try:
                        info_head, number, Fscore, CD1, CD2, name, _ = line.split('\t')
                        log_dict[method_name][name.strip()] = [float(CD2), float(Fscore)]
                    except:
                        info_head, number, Fscore, CD1, CD2, name = line.split('\t')
                        log_dict[method_name][name.strip()] = [float(CD2), float(Fscore)]
                else:   
                    try:
                        info_head, number, Fscore, CD1, CD2, EMD, name, _ = line.split('\t')
                        log_dict[method_name][name.strip()] = [float(CD2), float(Fscore)]
                    except:
                        info_head, number, Fscore, CD1, CD2, EMD, name = line.split('\t')
                        log_dict[method_name][name.strip()] = [float(CD2), float(Fscore)]
        # print(log_dict)
        method_name_list = method_name_list
        model_id_list = sorted(list(log_dict[method_name_list[0]].keys()))

        # output_file = 'PShapeNet55.txt'
        result_list = []

        for model_id in model_id_list:
            this_model_result = []
            for method in method_name_list:
                this_model_result.append(log_dict[method][model_id])
            result_list.append(this_model_result)
        result = np.array(result_list)
        # print(result.shape)
        result_for_mode.append(result)

    def convert_to_latex(array_list, skip_f1=False):
        str = ''
        
        for model_id, model in enumerate(model_id_list):
            str += f'{model}'
            for method_id, method in enumerate(method_name_list):
                for mode, array in enumerate(array_list):
                    performance = array[model_id]
                    best_cd_pos = performance[:, 0].argmin()
                    best_f1_pos = performance[:, 1].argmax()
                    # print(number)
                    if method_id == best_cd_pos:
                        str += '& \\textbf{' + f'{performance[method_id, 0]:.2f}' + '}'
                    else:
                        str += f'& {performance[method_id, 0]:.2f}'
                    if not skip_f1:
                        if method_id == best_f1_pos:
                            str += '& \\textbf{' + f'{performance[method_id, 1]:.2f}' + '}'
                        else:
                            str += f'& {performance[method_id, 1]:.3f}'
            str += '\\\\'
            str += '\n'
        return str

    result_latex = convert_to_latex(result_for_mode, skip_f1=True)
    with open(output_file, 'w') as f:
        f.write(result_latex)


In [3]:
produce_latex_table('./Log/ShapeNet55', 'ShapeNet55.txt', method_name_list=['Snowflake', 'AdaPoinTr'])
# produce_latex_table('./Log/UnseenShapeNet21', 'UnseenShapeNet21.txt', method_name_list=['Snowflake', 'AdaPoinTr'])


['2022-02-18 10:11:02,774 - SnowFlakeNet - INFO - 03759954', '112', '0.665 ', '12.464 ', '1.518 ', 'microphone', '\n']
['2022-02-18 10:11:02,774 - SnowFlakeNet - INFO - 04330267', '352', '0.359 ', '14.750 ', '0.750 ', 'stove', '\n']
['2022-02-18 10:11:02,774 - SnowFlakeNet - INFO - 03261776', '120', '0.411 ', '15.122 ', '0.940 ', 'earphone', '\n']
['2022-02-18 10:11:02,774 - SnowFlakeNet - INFO - 03513137', '264', '0.278 ', '18.544 ', '1.374 ', 'helmet', '\n']
['2022-02-18 10:11:02,774 - SnowFlakeNet - INFO - 04460130', '216', '0.499 ', '13.086 ', '0.680 ', 'tower', '\n']
['2022-02-18 10:11:02,775 - SnowFlakeNet - INFO - 04468005', '624', '0.451 ', '13.034 ', '0.639 ', 'train', '\n']
['2022-02-18 10:11:02,775 - SnowFlakeNet - INFO - 03761084', '248', '0.310 ', '15.396 ', '0.725 ', 'microwaves', '\n']
['2022-02-18 10:11:02,775 - SnowFlakeNet - INFO - 04004475', '272', '0.347 ', '15.070 ', '0.834 ', 'printer', '\n']
['2022-02-18 10:11:02,775 - SnowFlakeNet - INFO - 03938244', '160', '0.4